In [3]:
!python --version

Python 3.10.6


In [4]:
!pipenv install numpy pandas sklearn xgboost

Installing numpy...
Adding numpy to Pipfile's [packages]...
✔ Installation Succeeded 
Installing pandas...
Adding pandas to Pipfile's [packages]...
✔ Installation Succeeded 
Installing sklearn...
Adding sklearn to Pipfile's [packages]...
✔ Installation Succeeded 
Installing xgboost...
Adding xgboost to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (8b6b98) out of date, updating to (5fa943)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [dev-packages] dependencies...
Updated Pipfile.lock (25cc42be15df6c48ee4f5caa4d2260b8744448c39e5d2cf6b4d8dc6cf45fa943)!
Installing dependencies from Pipfile.lock (5fa943)...


In [11]:
!pipenv install bentoml

Installing bentoml...
Adding bentoml to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (5fa943) out of date, updating to (b8b89b)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [dev-packages] dependencies...
Updated Pipfile.lock (e4de6ba270d965989a34df61516a5b67fb949d0ad45c2fe93818431c69b8b89b)!
Installing dependencies from Pipfile.lock (b8b89b)...


In [1]:
!bentoml --version

bentoml, version 1.0.7


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

ModuleNotFoundError: No module named 'bentoml'

# Data preparation

In [15]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'
df = pd.read_csv(data)

In [16]:
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [17]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [18]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

# Random Forest

In [19]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=200,
                       random_state=1)

# XGBoost

Note:

We removed feature names

It was

features = dv.get_feature_names_out()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
Now it's

dtrain = xgb.DMatrix(X_train, label=y_train)

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [21]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

# Bento ML

In [3]:
import bentoml

ModuleNotFoundError: No module named 'bentoml'

In [4]:
!bentoml --version

bentoml, version 1.0.7


In [24]:
bentoml.xgboost.save_model(
    'credit_risk_model',
    model,
    custom_objects={
        'dictVectorizer': dv
    })

NameError: name 'bentoml' is not defined

Test

In [ ]:
import json

In [ ]:
request = df_test.iloc[0].to_dict()
print(json.dumps(request, indent=2))